In [20]:
import cv2 as cv
import numpy as np

img1=cv.imread('1.jpg')
gray1=cv.cvtColor(img1,cv.COLOR_BGR2GRAY)
img2=cv.imread('2.jpg')
gray2=cv.cvtColor(img2,cv.COLOR_BGR2GRAY)

sift=cv.SIFT_create()  # SIFT 알고리즘을 사용하여, 각 이미지에서 특징점과 디스크립터를 추출합니다.
kp1,des1=sift.detectAndCompute(gray1,None)
kp2,des2=sift.detectAndCompute(gray2,None)

flann_matcher=cv.DescriptorMatcher_create(cv.DescriptorMatcher_FLANNBASED) # FLANN(OpenCV의 빠른 최근접 이웃 검색) 알고리즘을 사용하여, 두 이미지의 디스크립터를 매칭
knn_match=flann_matcher.knnMatch(des1,des2,2)

T=0.7
good_match=[]
for nearest1,nearest2 in knn_match:  # knn 매칭 결과에서, 최근접 이웃과 두 번째로 가까운 이웃 간의 거리 비율이 T 이하인 경우를 좋은 매칭으로 판단
    if (nearest1.distance/nearest2.distance)<T:
        good_match.append(nearest1)

points1=np.float32([kp1[gm.queryIdx].pt for gm in good_match])
points2=np.float32([kp2[gm.trainIdx].pt for gm in good_match])
H,_=cv.findHomography(points1,points2,cv.RANSAC) # 좋은 매칭으로 선별된 특징점 좌표를 사용하여, RANSAC 기반 호모그래피 매트릭스를 구함

h1,w1=img1.shape[0],img1.shape[1]
h2,w2=img2.shape[0],img2.shape[1]

box1=np.float32([[0,0],[0,h1-1],[w1-1,h1-1],[w1-1,0]]).reshape(4,1,2)
box2=cv.perspectiveTransform(box1,H) # 호모그래피 매트릭스를 사용하여, img1의 변환 전 모서리 좌표를 img2의 좌표계로 변환

img2=cv.polylines(img2,[np.int32(box2)],True,(0,255,0),8)

img_match=np.empty((max(h1,h2),w1+w2,3),dtype=np.uint8) # 좋은 매칭 결과를 사용하여, 두 이미지를 이어 붙인 후, 매칭된 특징점을 그립니다.
cv.drawMatches(img1,kp1,img2,kp2,good_match,img_match,flags=cv.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)

cv.imshow('Matches and Homography',img_match)


k=cv.waitKey()
cv.destroyAllWindows()